<a href="https://colab.research.google.com/github/andreluisvsa/mlbsb/blob/master/Lesson10_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caderno de apoio para o vídeo 10 do curso de Machine Learning da Fast.ai - NLP

[Vídeo completo da lição 10 ](http://course.fast.ai/lessonsml1/lesson10.html)

## Link para Datasets utilizados, caderno do Jeremy e para exemplos do vídeo: 

[Dataset for English Words](https://www.kaggle.com/c/text-normalization-challenge-english-language)

[Exemplo do Kaggle](https://www.kaggle.com/alvira12/class-wise-processing-lb-0-992-new-dataset)

[Exemplo 2 do Kaggle](https://www.kaggle.com/neerjad/class-wise-regex-functions-l-b-0-995)

[Exemplo 3 do Kaggle ](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44629)

[Caderno do Jeremy para aula 10](https://github.com/fastai/fastai/blob/master/courses/ml1/lesson5-nlp.ipynb) - Lesson5NLP

[Caderno não-oficial para as aulas do Jeremy](https://forums.fast.ai/t/unofficial-lecture-10-notes/8538)

Vídeo sobre NLP e com o foco na aula 10 de fato começa a partir de 01:02:05 (Simple Linear Model)

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install fastai==0.7

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
#Editar entrada de dados
# Any results you write to the current directory are saved as output.

%reload_ext autoreload
%autoreload 2
#%matplotlib inline
 
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

Modelos lineares estão muito perto do estado da arte com o NLP - Jeremy 

Dataset do IMDB será utilizado para Sentiment Classification - Classificação do Sentimento dos Reviews dos filmes. Cerca de 50.000 reviews que contem classificações positivas e negativas. O dataset está dividido em treinamento e teste, com 25.000  reviews classificados para treinamento.  

Rodar os comandos no terminal para acesso ao Dataset: 

wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

gunzip aclImdb_v1.tar.gz

tar -xvf aclImdb_v1.tar


In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!gunzip aclImdb_v1.tar.gz
!tar -xvf aclImdb_v1.tar
#dataset = wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [0]:
PATH='aclImdb/'
names = ['neg','pos']

No vídeo ele da alguns passos para checar o dataset, mostrando os positivos e negativos, além de um "ls" no diretório para mostrar todos os arquivos disponíveis

In [0]:
trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
val,val_y = texts_labels_from_folders(f'{PATH}test',names)

In [0]:
texts_labels_from_folders??
#ele nao gosta de entrar nesse assunto, mas é interessante saber o que acontece na função que foi chamada acima. A funcao vai em cada diretorio e vai em cada arquivo no 
#diretorio e coloca no array de textos


No diretório positivo como exemplo, ele mostra um exemplo do arquivo de texto após a divisão em train/test

In [54]:
#ATIVIDADE: mostrar o primeiro item do train set, dentro da pasta


"The first look on the cover of this picture, it looks like a good rock n roll movie. But don't let the cover fool you, or the fact that Alice Cooper and Blondie is in it. The storyline is just horrible, and so is the acting. Plain and simple: BAD<br /><br />It's not a movie about a roadie, its just a thin love story, so awful that you see right through it. The only good thing about this movie, is the soundtrack.Some good songs, and that is why I give 2 out of 10. If it wasn't for the music, it would of been 0 out of 10. Meat Loaf is a horrible actor(at least he was in 1980), and the girl who plays the groupie isn't even good looking! This movie was a huge disappointment for me, because it makes a lot of good promises."

Após feito acima, ele mostra que na lista de train_y[0] não contem arquivo, pois ainda não foi classificado. 

In [55]:
trn_y[0]

0

Jeremy: "CountVectorizer converts a collection of text documents to a matrix of token counts (part of sklearn.feature_extraction.text)."

In [0]:
veczr = CountVectorizer(tokenizer=tokenize)
#sklearn - metodo para criar um vetor de palavras

Jeremy: 
"fit_transform(trn) finds the vocabulary in the training set. It also transforms the training set into a term-document matrix. Since we have to apply the same transformation to your validation set, the second line uses just the method transform(val). trn_term_doc and val_term_doc are sparse matrices. trn_term_doc[i] represents training document i and it contains a count of words for each document for each word in the vocabulary."

Vídeo: 01:06:50

Na explicação acima também é interessante salientar a etapa da Tokenização, que o Jeremy também explica de forma sensacional. Esta etapa é importante no pré-processamento de NLP. 

In [0]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [0]:
x=trn_term_doc
y=trn_y

Vídeo 01:12:50

In [62]:
trn_term_doc
#number of unique words - 25.000 
#

TypeError: ignored

In [61]:
print(trn_term_doc[0])
#93 daquelas palavras estao sendo usadas no primeiro documento. 

  (0, 52090)	1
  (0, 39532)	1
  (0, 40479)	1
  (0, 6430)	1
  (0, 41848)	1
  (0, 18637)	1
  (0, 32000)	1
  (0, 2)	1
  (0, 39416)	1
  (0, 22718)	1
  (0, 34676)	1
  (0, 28865)	1
  (0, 50418)	1
  (0, 73048)	1
  (0, 27577)	1
  (0, 10)	1
  (0, 411)	1
  (0, 72337)	2
  (0, 30228)	1
  (0, 38172)	1
  (0, 4721)	1
  (0, 9)	1
  (0, 1607)	1
  (0, 39168)	1
  (0, 41905)	1
  :	:
  (0, 25406)	1
  (0, 38507)	1
  (0, 65309)	3
  (0, 8)	4
  (0, 19558)	1
  (0, 9854)	1
  (0, 15)	8
  (0, 44202)	4
  (0, 56373)	1
  (0, 44788)	1
  (0, 56253)	1
  (0, 28081)	5
  (0, 1050)	7
  (0, 38797)	1
  (0, 39421)	1
  (0, 34716)	7
  (0, 13)	10
  (0, 49944)	1
  (0, 66684)	3
  (0, 46749)	5
  (0, 15180)	2
  (0, 46986)	1
  (0, 39408)	1
  (0, 24758)	1
  (0, 66458)	11


Com isso criamos uma matriz de vetores conhecida como Bag Of Words - BOW. Explicação do Jeremy é sensacional!

Vídeo 01:14:25

In [0]:
vocab = veczr.get_feature_names(); vocab[5000:5005] 
#exemplo de alguns dos elementos 

In [0]:
w0 = set([o.lower() for o in trn[0].split(' ')]); w0

Vídeo 01:14:48 

Jeremy explica que não utilizou a função de tokenização do fastai ou do sklearn, mas mostra como ele criou esse tipo de função usando list comprehension.
No final, ele tem 91 tokens, diferente do 93 citados acima. E isso é pelo fato dele NÃO ter utilizado o tokenizer. 

In [0]:
len(w0)

In [0]:
veczr.vocabulary_['absurd']
#aqui ele checa o ID de uma palavra em particular, que deve retornar 1297

In [0]:
trn_term_doc[0,1297]
#resultado deve ser quantas vezes essa palavra apareceu naquele documento - 2!

In [0]:
trn_term_doc[0,5000]
#0

## Naive Bayes

Jeremy: "Bag of Word approach"

We define the log-count ratio $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive document has a feature divided by the number of positive documents

In [0]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

Vídeo: 01:17:28

Jeremy explica muito bem no Excel primeiramente, depois faz o modelo no caderno. 

Modelo probabilistico. Machine Learning, a column is a feature. 

Pode ser feito conforme caderno (que utiliza a função acima):

In [0]:
x=trn_term_doc
y=trn_y

r = np.log(pr(1)/pr(0))
b = np.log((y==1).mean() / (y==0).mean())

Ou, conforme vídeo (01:26:50):

In [0]:
x=trn_term_doc
y=trn_y

p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1

r = np.log(p/p.sum())/(q/q.sum())
b = np.log(len(p)/len(q))

Fórmula para o Naive Bayes

In [0]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

Vídeo 01:29:00

E para o Naive Bayes de forma binarizada... 

In [0]:
x=trn_term_doc.sign()
r = np.log(pr(1)/pr(0))

pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

## Logistic Regression


Agora vamos passar para um modelo um pouco melhor de avaliação - Vídeo 01:30:05

In [0]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

#a partir do calculo com regressão logistica - teórico - o resultado final é melhor do que o método anterior. 

E uma versão binarizada da Regressão Logística (Vídeo 01:32:20)

In [0]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

E uma versão regularizada...

In [0]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

E uma versão binarizada da versão regularizada...

In [0]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

Explicação geral: Vídeo 01:33:45

## Trigram com Naive Bayes


Vídeo: 01:35:40


Jeremy:

Our next model is a version of logistic regression with Naive Bayes features described here. For every document we compute binarized features as described above, but this time we use bigrams and trigrams too. Each feature is a log-count ratio. A logistic regression model is then trained to predict sentiment.

O Sklearn por padrão começa o modelo de vectorização com UniGrams. Podemos alterar a configuração para ele entender BiGrams e TriGrams. (ngram_rang(1,3))



In [0]:
veczr =  CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=800000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [0]:
trn_term_doc.shape

In [0]:
vocab = veczr.get_feature_names()

In [0]:
vocab[200000:200005]

Apresenta melhores resultados, como o exemplo mostrado no vídeo "by vast" e "by vera miles"

Conforme vídeo:

In [0]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()

p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1

r = np.log(p/p.sum())/(q/q.sum())
b = np.log(len(p)/len(q))

Ou conforme caderno:

In [0]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()

In [0]:
r = np.log(pr(1) / pr(0))
b = np.log((y==1).mean() / (y==0).mean())

E aqui fazemos o ajuste para a Regressão Logística com os TriGrams 

In [0]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

Continua no próximo vídeo...
